In [2]:
from haystack.telemetry import tutorial_running
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
import logging
from haystack.nodes import PreProcessor
from langdetect import detect
import nltk
import pandas as pd
import numpy as np



c:\Users\sotof\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Logs
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [5]:
# Specify the file path to your CSV file
file_path = "dominio3Test.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df.head()

,PageURL,Title,CreationDate,Seccion,Subseccion,Contenido,WikipageID,LastModified
0,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,NaN,NaN,Carbon dioxide is a chemical compound with the...,5906,2023-09-25T06:26:03Z
1,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,"Structure, bonding and molecular vibrations",The symmetry of a carbon dioxide molecule is l...,5906,2023-09-25T06:26:03Z
2,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,In aqueous solution,"Carbon dioxide is soluble in water, in which i...",5906,2023-09-25T06:26:03Z
3,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,Chemical reactions of CO2,CO2 is a potent electrophile having an electro...,5906,2023-09-25T06:26:03Z
4,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,Physical properties,Carbon dioxide is colorless. At low concentrat...,5906,2023-09-25T06:26:03Z


In [6]:
# Extraer las primeras 'numero_de_filas_a_extraer' filas y almacenarlas en un nuevo DataFrame
df_test = df.head(6)
# Add a new column 'id' starting from 1 at the first position
df_test.insert(0, 'id', range(1, len(df_test) + 1))
df_test.head()


df_test

,PageURL,Title,CreationDate,Seccion,Subseccion,Contenido,WikipageID,LastModified
0,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,NaN,NaN,Carbon dioxide is a chemical compound with the...,5906,2023-09-25T06:26:03Z
1,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,"Structure, bonding and molecular vibrations",The symmetry of a carbon dioxide molecule is l...,5906,2023-09-25T06:26:03Z
2,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,In aqueous solution,"Carbon dioxide is soluble in water, in which i...",5906,2023-09-25T06:26:03Z
3,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,Chemical reactions of CO2,CO2 is a potent electrophile having an electro...,5906,2023-09-25T06:26:03Z
4,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,Physical properties,Carbon dioxide is colorless. At low concentrat...,5906,2023-09-25T06:26:03Z
5,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Biological role,Physical properties,Carbon dioxide is an end product of cellular r...,5906,2023-09-25T06:26:03Z


In [7]:
import io

# Specify the row and column you want to extract
row_index = 1  # Index of the row (e.g., 2 for 'Charlie')
column_name = 'Contenido'  # Name of the column


# Extract the cell content
cell_content = df_test.at[row_index, column_name]

# Create an in-memory file
memory_file = io.StringIO()

# Write text to the file
memory_file.write(cell_content)

# Get the contents of the file
# file_contents = memory_file.getvalue()

# Close the in-memory file (optional, but good practice)
# memory_file.close()

# Print the contents
#print(file_contents)

2009

In [9]:
import pandas as pd

# Especifica el índice de fila y la columna que deseas extraer
fila = 0  # Por ejemplo, extraeremos la fila 2 (índice 2)
columna = 'Contenido'

# Almacenar el contenido de la celda en una variable
contenido = df_test.at[fila, columna]

# Crear el archivo TXT y escribir el contenido
nombre_archivo = "haystack-division/ContenidoCelda.txt"
with open(nombre_archivo, 'w', encoding='UTF-8') as archivo:
    archivo.write(contenido)

print(f"El archivo {nombre_archivo} se ha creado y el contenido se ha almacenado correctamente.")



El archivo haystack-division/ContenidoCelda.txt se ha creado y el contenido se ha almacenado correctamente.


In [ ]:
from haystack.nodes import PreProcessor


# This is a default usage of the PreProcessor.
# Here, it performs cleaning of consecutive whitespaces
# and splits a single large document into smaller documents.
# Each document is up to 1000 words long and document breaks cannot fall in the middle of sentences
# Note how the single document passed into the document gets split into 5 smaller documents

preprocessor = PreProcessor(

    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by= "sentence",
    split_length = 5,
    split_respect_sentence_boundary=False

)

In [ ]:
def createHaystackArchive(fila, columna):

    try:

        # Especifica el índice de fila y la columna que deseas extraer
        fila = 0  # Por ejemplo, extraeremos la fila 2 (índice 2)
        columna = 'Contenido'

        # Almacenar el contenido de la celda en una variable
        contenido = df_test.at[fila, columna]

        # Crear el archivo TXT y escribir el contenido
        nombre_archivo = "haystack-division/ContenidoCelda.txt"
        with open(nombre_archivo, 'w', encoding='UTF-8') as archivo:
            archivo.write(contenido)

        print(f"El archivo {nombre_archivo} se ha creado y el contenido se ha almacenado correctamente.")

    except Exception as e:
        print(f"An error occurred: {e}")

    

In [20]:
from transformers import AutoTokenizer

def lenghtToken(text, llm):
    tokenizer = AutoTokenizer.from_pretrained(llm)
    #Tokenize the input text
    tokens = tokenizer.tokenize(text)

    return len(tokens)

In [21]:
text = 'Hi how are'
llm = 'google/flan-t5-base'

print(lenghtToken(text,llm))

3


In [10]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
from haystack.utils import convert_files_to_docs

for i in range(len(df_test)):

    # Especifica el índice de fila y la columna que deseas extraer
    fila = i  # Por ejemplo, extraeremos la fila 2 (índice 2)
    columna = 'Contenido'

    # Almacenar el contenido de la celda en una variable
    contenido = df_test.at[fila, columna]

    # Crear el archivo TXT y escribir el contenido
    nombre_archivo = "haystack-division/ContenidoCelda.txt"
    with open(nombre_archivo, 'w', encoding='UTF-8') as archivo:
        archivo.write(contenido)

    print(f"El archivo {nombre_archivo} se ha creado y el contenido se ha almacenado correctamente.")

    converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
    doc_txt = converter.convert(file_path="haystack-division\ContenidoCelda.txt", meta=None)[0]

    
    doc_dir = 'haystack-division'

    all_docs = convert_files_to_docs(dir_path=doc_dir)


    docs_default = preprocessor.process([doc_txt])
    print(f"n_docs_input: 1\nn_docs_output: {len(docs_default)}")

    # Define an array of strings
    fragments_list = []
    split_id_list = []

    # Specify the target word you want to search for
    #target_word = 'content'

    # Iterate through the array elements
    for element in docs_default:

        #print(element)

        document_dict = element.to_dict()
        #print(document_dict)

        for key, value in document_dict.items():

            if (key == 'content'):
                
                #print(f'{key}, {value}')
                fragments_list.append(value)
            
            elif (key == 'meta'):

                for key_meta, value_meta in value.items():

                    #print(f'{key_meta} === {value_meta}')
                    #print(value_meta)
                    split_id_list.append(value_meta)
        

        # Determine the number of rows to add based on the maximum length of the lists
    num_rows_to_add = max(len(fragments_list), len(split_id_list))

    # Create an empty DataFrame with the same columns as the original DataFrame
    new_columns = ['split', 'split_id']
    new_df = pd.DataFrame(columns=df_test.columns.tolist() + new_columns)

    # Fill the new columns with data from the lists
    new_df['split'] = fragments_list[:num_rows_to_add]
    new_df['split_id'] = split_id_list[:num_rows_to_add]

    # Fill the other columns with NaN values
    for col in new_df.columns:
        if col not in ['id'] + new_columns:
            new_df[col] = np.nan

    id_value = 

    # Use the "id" column to fill the NaN values in the remaining columns
    for index, row in new_df.iterrows():

        original_row = df_test[df_test['id'] == id_value]
        
        if not original_row.empty:
        
            for col in original_row.columns:
                
                if col not in ['id']:
                    
                    new_df.at[index, col] = original_row[col].values[0]





0
1
2
3
4
5


In [9]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_txt = converter.convert(file_path="haystack-division\ContenidoCelda.txt", meta=None)[0]

In [10]:
from haystack.utils import convert_files_to_docs

doc_dir = 'haystack-division'

all_docs = convert_files_to_docs(dir_path=doc_dir)

INFO - haystack.utils.preprocessing -  Converting haystack-division\ContenidoCelda.txt


In [11]:
from haystack.nodes import PreProcessor


# This is a default usage of the PreProcessor.
# Here, it performs cleaning of consecutive whitespaces
# and splits a single large document into smaller documents.
# Each document is up to 1000 words long and document breaks cannot fall in the middle of sentences
# Note how the single document passed into the document gets split into 5 smaller documents

preprocessor = PreProcessor(

    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by= "sentence",
    split_length = 5,
    split_respect_sentence_boundary=False

)


In [12]:
docs_default = preprocessor.process([doc_txt])
print(f"n_docs_input: 1\nn_docs_output: {len(docs_default)}")

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 75.34docs/s]

n_docs_input: 1
n_docs_output: 4


In [13]:
print(docs_default)

[<Document: {'content': 'Carbon dioxide is a chemical compound with the chemical formula CO2. It is made up of molecules that each have one carbon atom covalently double bonded to two oxygen atoms. It is found in the gas state at room temperature, and as the source of available carbon in the carbon cycle, atmospheric CO2 is the primary carbon source for life on Earth. In the air, carbon dioxide is transparent to visible light but absorbs infrared radiation, acting as a greenhouse gas. Carbon dioxide is soluble in water and is found in groundwater, lakes, ice caps, and seawater. ', 'content_type': 'text', 'score': None, 'meta': {'_split_id': 0}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '75c7d8c5eec1488adab5309e48fa273b'}>, <Document: {'content': "When carbon dioxide dissolves in water, it forms carbonate and mainly bicarbonate (HCO−3), which causes ocean acidification as atmospheric CO2 levels increase.It is a trace gas in Earth's atmosphere at 421 parts per million (ppm), 

In [14]:
document = docs_default[0]
document_dict = document.to_dict()

# print(document_json)

print(document_dict)

{'content': 'Carbon dioxide is a chemical compound with the chemical formula CO2. It is made up of molecules that each have one carbon atom covalently double bonded to two oxygen atoms. It is found in the gas state at room temperature, and as the source of available carbon in the carbon cycle, atmospheric CO2 is the primary carbon source for life on Earth. In the air, carbon dioxide is transparent to visible light but absorbs infrared radiation, acting as a greenhouse gas. Carbon dioxide is soluble in water and is found in groundwater, lakes, ice caps, and seawater. ', 'content_type': 'text', 'score': None, 'meta': {'_split_id': 0}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '75c7d8c5eec1488adab5309e48fa273b'}


In [15]:
# Define an array of strings
fragments_list = []
split_id_list = [] 

In [16]:
# Specify the target word you want to search for
target_word = 'content'

# Iterate through the array elements
for element in docs_default:

    #print(element)

    document_dict = element.to_dict()
    #print(document_dict)

    for key, value in document_dict.items():

        if (key == 'content'):
            
            #print(f'{key}, {value}')
            fragments_list.append(value)
        
        elif (key == 'meta'):

            for key_meta, value_meta in value.items():

                #print(f'{key_meta} === {value_meta}')
                #print(value_meta)
                split_id_list.append(value_meta)

    
    # Determine the number of rows to add based on the maximum length of the lists
    num_rows_to_add = max(len(fragments_list), len(split_id_list))

    # Create an empty DataFrame with the same columns as the original DataFrame
    new_columns = ['split', 'split_id']
    new_df = pd.DataFrame(columns=df_test.columns.tolist() + new_columns)

    # Fill the new columns with data from the lists
    new_df['split'] = fragments_list[:num_rows_to_add]
    new_df['split_id'] = split_id_list[:num_rows_to_add]

    # Fill the other columns with NaN values
    for col in new_df.columns:
        if col not in ['id'] + new_columns:
            new_df[col] = np.nan

    id_value = 1

    # Use the "id" column to fill the NaN values in the remaining columns
    for index, row in new_df.iterrows():

        original_row = df_test[df_test['id'] == id_value]
        
        if not original_row.empty:
        
            for col in original_row.columns:
                
                if col not in ['id']:
                    
                    new_df.at[index, col] = original_row[col].values[0]




In [17]:
print(fragments_list)
print('===================')
print(split_id_list)
print('===================')
print(f'matrix_fragments[0][1]')

['Carbon dioxide is a chemical compound with the chemical formula CO2. It is made up of molecules that each have one carbon atom covalently double bonded to two oxygen atoms. It is found in the gas state at room temperature, and as the source of available carbon in the carbon cycle, atmospheric CO2 is the primary carbon source for life on Earth. In the air, carbon dioxide is transparent to visible light but absorbs infrared radiation, acting as a greenhouse gas. Carbon dioxide is soluble in water and is found in groundwater, lakes, ice caps, and seawater. ', "When carbon dioxide dissolves in water, it forms carbonate and mainly bicarbonate (HCO−3), which causes ocean acidification as atmospheric CO2 levels increase.It is a trace gas in Earth's atmosphere at 421 parts per million (ppm), or about 0.04% (as of May 2022) having risen from pre-industrial levels of 280 ppm or about 0.025%. Burning fossil fuels is the primary cause of these increased CO2 concentrations and also the primary ca

In [18]:
# Add a new column 'id' starting from 1 at the first position
df_test.insert(0, 'id', range(1, len(df_test) + 1))
df_test.head()

,id,PageURL,Title,CreationDate,Seccion,Subseccion,Contenido,WikipageID,LastModified
0,1,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,NaN,NaN,Carbon dioxide is a chemical compound with the...,5906,2023-09-25T06:26:03Z
1,2,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,"Structure, bonding and molecular vibrations",The symmetry of a carbon dioxide molecule is l...,5906,2023-09-25T06:26:03Z
2,3,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,In aqueous solution,"Carbon dioxide is soluble in water, in which i...",5906,2023-09-25T06:26:03Z
3,4,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,Chemical reactions of CO2,CO2 is a potent electrophile having an electro...,5906,2023-09-25T06:26:03Z
4,5,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,Physical properties,Carbon dioxide is colorless. At low concentrat...,5906,2023-09-25T06:26:03Z


In [27]:
# Determine the number of rows to add based on the length of the lists
num_rows_to_add = max(len(fragments_list), len(split_id_list))
print(f'Buenas tardes = {num_rows_to_add}')


Buenas tardes = 4


In [36]:
'''
# Define an array of strings
fragments_list = []
split_id_list = [] 

'''
# Determine the number of rows to add based on the maximum length of the lists
num_rows_to_add = max(len(fragments_list), len(split_id_list))

# Create an empty DataFrame with the same columns as the original DataFrame
new_columns = ['split', 'split_id']
new_df = pd.DataFrame(columns=df_test.columns.tolist() + new_columns)

# Fill the new columns with data from the lists
new_df['split'] = fragments_list[:num_rows_to_add]
new_df['split_id'] = split_id_list[:num_rows_to_add]

# Fill the other columns with NaN values
for col in new_df.columns:
    if col not in ['id'] + new_columns:
        new_df[col] = np.nan

id_value = 1

# Use the "id" column to fill the NaN values in the remaining columns
for index, row in new_df.iterrows():
    original_row = df_test[df_test['id'] == id_value]
    
    if not original_row.empty:
       
       for col in original_row.columns:
           
           if col not in ['id']:
               
               new_df.at[index, col] = original_row[col].values[0]
    


In [37]:
new_df



,id,PageURL,Title,CreationDate,Seccion,Subseccion,Contenido,WikipageID,LastModified,split,split_id
0,NaN,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,NaN,NaN,Carbon dioxide is a chemical compound with the...,5906.0,2023-09-25T06:26:03Z,Carbon dioxide is a chemical compound with the...,0
1,NaN,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,NaN,NaN,Carbon dioxide is a chemical compound with the...,5906.0,2023-09-25T06:26:03Z,"When carbon dioxide dissolves in water, it for...",1
2,NaN,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,NaN,NaN,Carbon dioxide is a chemical compound with the...,5906.0,2023-09-25T06:26:03Z,"Since plants require CO2 for photosynthesis, a...",2
3,NaN,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,NaN,NaN,Carbon dioxide is a chemical compound with the...,5906.0,2023-09-25T06:26:03Z,Sequestered CO2 is released into the atmospher...,3
